In [1]:
import warnings

warnings.filterwarnings("ignore")
import pandas as pd
import pertpy as pt

from sccoda.util import comp_ana as mod
from sccoda.util import cell_composition_data as dat

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


## Load data

In [2]:
# Load data
haber_cells = pt.dt.haber_2017_regions()
# Extract condition from mouse name and add it as an extra column to the covariates
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(
    haber_cells,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="cell_label",
    sample_identifier="batch",
    covariate_obs=["condition"],
)
sccoda_data.mod["coda_salm"] = sccoda_data["coda"][
    sccoda_data["coda"].obs["condition"].isin(["Control", "Salmonella"])
].copy()

# Select control and salmonella data
data_salm = sccoda_data.mod["coda_salm"]

## Basic model

### Old implementation

In [14]:
# run model
model_old = mod.CompositionalAnalysis(
    data_salm, formula="condition", reference_cell_type="Goblet"
)
# run mcmc
results_old = model_old.sample_hmc()

# show results
results_old.summary()
print(results_old.credible_effects())
results_old.set_fdr(est_fdr=0.4)
results_old.summary()

100%|██████████| 20000/20000 [00:26<00:00, 766.46it/s]


MCMC sampling finished. (33.233 sec)
Acceptance rate: 60.8%
Compositional Analysis summary:

Data: 6 samples, 8 cell types
Reference index: 3
Formula: condition

Intercepts:
                       Final Parameter  Expected Sample
Cell Type                                              
Endocrine                        1.115        34.514313
Enterocyte                       2.332       116.556264
Enterocyte.Progenitor            2.522       140.945615
Goblet                           1.751        65.194435
Stem                             2.703       168.911364
TA                               2.114        93.725857
TA.Early                         2.861       197.823280
Tuft                             0.426        17.328874


Effects:
                                               Final Parameter  \
Covariate               Cell Type                                
condition[T.Salmonella] Endocrine                     0.000000   
                        Enterocyte                    1.3

### New implementation

In [4]:
sccoda_data = sccoda_model.prepare(
    sccoda_data,
    modality_key="coda_salm",
    formula="condition",
    reference_cell_type="Goblet",
)

# Run MCMC
sccoda_model.run_nuts(sccoda_data, modality_key="coda_salm")

# show results
sccoda_model.summary(sccoda_data, modality_key="coda_salm")
sccoda_model.credible_effects(sccoda_data, modality_key="coda_salm")
sccoda_model.set_fdr(sccoda_data, modality_key="coda_salm", est_fdr=0.4)
sccoda_model.summary(sccoda_data, modality_key="coda_salm")

sample: 100%|██████████| 11000/11000 [00:13<00:00, 819.51it/s, 127 steps of size 3.69e-02. acc. prob=0.70]


                                          Compositional Analysis summary                                           
┌──────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────┐
│ Name                                         │ Value                                                            │
├──────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────┤
│ Data                                         │ Data: 6 samples, 8 cell types                                    │
│ Reference cell type                          │ Goblet                                                           │
│ Formula                                      │ condition                                                        │
└──────────────────────────────────────────────┴──────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Intercepts                                                                                                      │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                        Final Parameter  Expected Sample                                                         │
│ Cell Type                                                                                                       │
│ Endocrine                  1.106            34.617                                                              │
│ Enterocyte                 2.322           116.788                                                              │
│ Enterocyte.Progenitor      2.507           140.521                                                              │
│ Goblet                     1.739            65.193                                                              │
│ Stem                       2.692           169.077                                                              │
│ TA                         2.103            93.818                                                              │
│ TA.Early                   2.848           197.622                                                              │
│ Tuft                       0.416            17.363                                                              │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Effects                                                                                                         │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                                              Final Parameter  Expected Sample  log2-fold change                 │
│ Covariate             Cell Type                                                                                 │
│ conditionT.Salmonella Endocrine                  0.000            24.709            -0.486                      │
│                       Enterocyte                 1.353           322.368             1.465                      │
│                       Enterocyte.Progenitor      0.000           100.298            -0.486                      │
│                       Goblet                     0.000            46.532            -0.486                      │
│                       Stem                       0.000           120.681            -0.486                      │
│                       TA                         0.000            66.964            -0.486                      │
│                       TA.Early                   0.000           141.055            -0.486                      │
│                       Tuft                       0.000            12.393            -0.486                      │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

                                          Compositional Analysis summary                                           
┌──────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────┐
│ Name                                         │ Value                                                            │
├──────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────┤
│ Data                                         │ Data: 6 samples, 8 cell types                                    │
│ Reference cell type                          │ Goblet                                                           │
│ Formula                                      │ condition                                                        │
└──────────────────────────────────────────────┴──────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Intercepts                                                                                                      │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                        Final Parameter  Expected Sample                                                         │
│ Cell Type                                                                                                       │
│ Endocrine                  1.106            34.617                                                              │
│ Enterocyte                 2.322           116.788                                                              │
│ Enterocyte.Progenitor      2.507           140.521                                                              │
│ Goblet                     1.739            65.193                                                              │
│ Stem                       2.692           169.077                                                              │
│ TA                         2.103            93.818                                                              │
│ TA.Early                   2.848           197.622                                                              │
│ Tuft                       0.416            17.363                                                              │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Effects                                                                                                         │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                                              Final Parameter  Expected Sample  log2-fold change                 │
│ Covariate             Cell Type                                                                                 │
│ conditionT.Salmonella Endocrine                  0.284            32.516            -0.090                      │
│                       Enterocyte                 1.353           319.204             1.451                      │
│                       Enterocyte.Progenitor      0.000            99.314            -0.501                      │
│                       Goblet                     0.000            46.076            -0.501                      │
│                       Stem                       0.000           119.497            -0.501                      │
│                       TA                         0.000            66.306            -0.501                      │
│                       TA.Early                   0.000           139.671            -0.501                      │
│                       Tuft                       0.012            12.416            -0.484                      │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

### Compare

In [23]:
result_df_old = results_old.effect_df
result_df_new = sccoda_model.get_effect_df(sccoda_data["coda_salm"])

result_df_combined = pd.DataFrame({
    "Final Parameter (old)": result_df_old["Final Parameter"].values,
    "Final Parameter (new)": result_df_new["Final Parameter"],
    "log2-fold change (old)": result_df_old["log2-fold change"].values,
    "log2-fold change (new)": result_df_new["log2-fold change"],
},index=result_df_new.index)
result_df_combined

Final Parameter (old)  \
Covariate             Cell Type                                      
conditionT.Salmonella Endocrine                           0.283754   
                      Enterocyte                          1.346851   
                      Enterocyte.Progenitor               0.000000   
                      Goblet                              0.000000   
                      Stem                                0.000000   
                      TA                                  0.000000   
                      TA.Early                            0.000000   
                      Tuft                                0.019015   

                                             Final Parameter (new)  \
Covariate             Cell Type                                      
conditionT.Salmonella Endocrine                           0.284457   
                      Enterocyte                          1.352542   
                      Enterocyte.Progenitor               0.000000   
                      Goblet                              0.000000   
                      Stem                                0.000000   
                      TA                                  0.000000   
                      TA.Early                            0.000000   
                      Tuft                                0.011722   

                                             log2-fold change (old)  \
Covariate             Cell Type                                       
conditionT.Salmonella Endocrine                           -0.087481   
                      Enterocyte                           1.446244   
                      Enterocyte.Progenitor               -0.496852   
                      Goblet                              -0.496852   
                      Stem                                -0.496852   
                      TA                                  -0.496852   
                      TA.Early                            -0.496852   
                      Tuft                                -0.469419   

                                             log2-fold change (new)  
Covariate             Cell Type                                      
conditionT.Salmonella Endocrine                           -0.090331  
                      Enterocyte                           1.450591  
                      Enterocyte.Progenitor               -0.500715  
                      Goblet                              -0.500715  
                      Stem                                -0.500715  
                      TA                                  -0.500715  
                      TA.Early                            -0.500715  
                      Tuft                                -0.483803

## Multigroup

### Old implementation

In [24]:
data_all = sccoda_data.mod["coda"]
model_all = mod.CompositionalAnalysis(
    data_all, formula="condition", reference_cell_type="Endocrine"
)
all_results = model_all.sample_hmc()
all_results.summary()

100%|██████████| 20000/20000 [00:32<00:00, 620.88it/s]


MCMC sampling finished. (41.133 sec)
Acceptance rate: 55.7%
Compositional Analysis summary:

Data: 10 samples, 8 cell types
Reference index: 0
Formula: condition

Intercepts:
                       Final Parameter  Expected Sample
Cell Type                                              
Endocrine                        1.004        47.032938
Enterocyte                       1.925       118.137378
Enterocyte.Progenitor            2.352       181.063567
Goblet                           1.475        75.327718
Stem                             2.428       195.360813
TA                               1.883       113.278362
TA.Early                         2.549       220.489080
Tuft                             0.665        33.510143


Effects:
                                                Final Parameter  \
Covariate                Cell Type                                
condition[T.Hpoly.Day3]  Endocrine                     0.000000   
                         Enterocyte                  

### New implementation

In [25]:
# model all three diseases at once
sccoda_data = sccoda_model.prepare(
    sccoda_data,
    modality_key="coda",
    formula="condition",
    reference_cell_type="Endocrine",
)
sccoda_model.run_nuts(sccoda_data, modality_key="coda")
sccoda_model.summary(sccoda_data, modality_key="coda")

sample: 100%|██████████| 11000/11000 [00:34<00:00, 323.31it/s, 255 steps of size 2.07e-02. acc. prob=0.74]


                                          Compositional Analysis summary                                           
┌─────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────┐
│ Name                                        │ Value                                                             │
├─────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────┤
│ Data                                        │ Data: 10 samples, 8 cell types                                    │
│ Reference cell type                         │ Endocrine                                                         │
│ Formula                                     │ condition                                                         │
└─────────────────────────────────────────────┴───────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Intercepts                                                                                                      │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                        Final Parameter  Expected Sample                                                         │
│ Cell Type                                                                                                       │
│ Endocrine                  0.959            46.219                                                              │
│ Enterocyte                 1.882           116.326                                                              │
│ Enterocyte.Progenitor      2.333           182.618                                                              │
│ Goblet                     1.481            77.898                                                              │
│ Stem                       2.398           194.882                                                              │
│ TA                         1.861           113.909                                                              │
│ TA.Early                   2.512           218.415                                                              │
│ Tuft                       0.650            33.933                                                              │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Effects                                                                                                         │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                                               Final Parameter  Expected Sample  log2-fold change                │
│ Covariate              Cell Type                                                                                │
│ conditionT.Hpoly.Day3  Endocrine                  0.000            46.219             0.000                     │
│                        Enterocyte                 0.000           116.326             0.000                     │
│                        Enterocyte.Progenitor      0.000           182.618             0.000                     │
│                        Goblet                     0.000            77.898             0.000                     │
│                        Stem                       0.000           194.882             0.000                     │
│                        TA                         0.000           113.909             0.000                     │
│                        TA.Early                   0.000           218.415             0.000                     │
│                        Tuft                       0.000            33.933             0.000                     │
│ conditionT.Hpoly.Day10 Endocrine                  0.000            46.219             0.000                     │
│                        Enterocyte                 0.000           116.326             0.000                     │
│                        Enterocyte.Progenitor      0.000           182.618             0.000                     │
│                        Goblet                     0.000            77.898             0.000                     │
│                        Stem                       0.000           194.882             0.000                     │
│                        TA                         0.000           113.909             0.000                     │
│                        TA.Early                   0.000           218.415             0.000                     │
│                        Tuft                       0.000            33.933             0.000                     │
│ conditionT.Salmonella  Endocrine                  0.000            32.004            -0.530                     │
│                        Enterocyte                 1.560           383.256             1.720                     │
│                        Enterocyte.Progenitor      0.000           126.450            -0.530                     │
│                        Goblet                     0.000            53.939            -0.530                     │
│                        Stem                       0.000           134.943            -0.530                     │
│                        TA                         0.000            78.874            -0.530                     │
│                        TA.Early                   0.000           151.237            -0.530                     │
│                        Tuft                       0.000            23.496            -0.530                     │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

### Compare

In [26]:
result_df_old = all_results.effect_df
result_df_new = sccoda_model.get_effect_df(sccoda_data["coda"])

result_df_combined = pd.DataFrame({
    "Final Parameter (old)": result_df_old["Final Parameter"].values,
    "Final Parameter (new)": result_df_new["Final Parameter"],
    "log2-fold change (old)": result_df_old["log2-fold change"].values,
    "log2-fold change (new)": result_df_new["log2-fold change"],
},index=result_df_new.index)
result_df_combined

Final Parameter (old)  \
Covariate              Cell Type                                      
conditionT.Hpoly.Day3  Endocrine                           0.000000   
                       Enterocyte                          0.000000   
                       Enterocyte.Progenitor               0.000000   
                       Goblet                              0.000000   
                       Stem                                0.000000   
                       TA                                  0.000000   
                       TA.Early                            0.000000   
                       Tuft                                0.000000   
conditionT.Hpoly.Day10 Endocrine                           0.000000   
                       Enterocyte                          0.000000   
                       Enterocyte.Progenitor               0.000000   
                       Goblet                              0.000000   
                       Stem                                0.000000   
                       TA                                  0.000000   
                       TA.Early                            0.000000   
                       Tuft                                0.000000   
conditionT.Salmonella  Endocrine                           0.000000   
                       Enterocyte                          1.539127   
                       Enterocyte.Progenitor               0.000000   
                       Goblet                              0.000000   
                       Stem                                0.000000   
                       TA                                  0.000000   
                       TA.Early                            0.000000   
                       Tuft                                0.000000   

                                              Final Parameter (new)  \
Covariate              Cell Type                                      
conditionT.Hpoly.Day3  Endocrine                           0.000000   
                       Enterocyte                          0.000000   
                       Enterocyte.Progenitor               0.000000   
                       Goblet                              0.000000   
                       Stem                                0.000000   
                       TA                                  0.000000   
                       TA.Early                            0.000000   
                       Tuft                                0.000000   
conditionT.Hpoly.Day10 Endocrine                           0.000000   
                       Enterocyte                          0.000000   
                       Enterocyte.Progenitor               0.000000   
                       Goblet                              0.000000   
                       Stem                                0.000000   
                       TA                                  0.000000   
                       TA.Early                            0.000000   
                       Tuft                                0.000000   
conditionT.Salmonella  Endocrine                           0.000000   
                       Enterocyte                          1.559854   
                       Enterocyte.Progenitor               0.000000   
                       Goblet                              0.000000   
                       Stem                                0.000000   
                       TA                                  0.000000   
                       TA.Early                            0.000000   
                       Tuft                                0.000000   

                                              log2-fold change (old)  \
Covariate              Cell Type                                       
conditionT.Hpoly.Day3  Endocrine                            0.000000   
                       Enterocyte                           0.000000   
                       Enterocyte.Progenitor                0.00

## Different base label

### Old implementation

In [27]:
model_salm_switch_cond = mod.CompositionalAnalysis(
    data_salm, formula="C(condition, Treatment('Salmonella'))", reference_cell_type="Goblet"
)
switch_results = model_salm_switch_cond.sample_hmc()
switch_results.summary()

100%|██████████| 20000/20000 [00:26<00:00, 751.89it/s]


MCMC sampling finished. (33.783 sec)
Acceptance rate: 67.2%
Compositional Analysis summary:

Data: 6 samples, 8 cell types
Reference index: 3
Formula: C(condition, Treatment('Salmonella'))

Intercepts:
                       Final Parameter  Expected Sample
Cell Type                                              
Endocrine                        1.224        27.904021
Enterocyte                       3.681       325.632946
Enterocyte.Progenitor            2.547       104.770350
Goblet                           1.751        47.265036
Stem                             2.607       111.248987
TA                               2.031        62.537778
TA.Early                         2.858       142.989445
Tuft                             0.433        12.651437


Effects:
                                                                        Final Parameter  \
Covariate                                        Cell Type                                
C(condition, Treatment('Salmonella'))[T.Contr

### New implementation

In [28]:
sccoda_data = sccoda_model.prepare(
    sccoda_data,
    modality_key="coda_salm",
    formula="C(condition, Treatment('Salmonella'))",
    reference_cell_type="Goblet",
)
sccoda_model.run_nuts(sccoda_data, modality_key="coda_salm")
sccoda_model.summary(sccoda_data, modality_key="coda_salm")

sample: 100%|██████████| 11000/11000 [00:26<00:00, 419.77it/s, 127 steps of size 1.86e-02. acc. prob=0.88]


                                          Compositional Analysis summary                                           
┌────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────┐
│ Name                                   │ Value                                                                  │
├────────────────────────────────────────┼────────────────────────────────────────────────────────────────────────┤
│ Data                                   │ Data: 6 samples, 8 cell types                                          │
│ Reference cell type                    │ Goblet                                                                 │
│ Formula                                │ C(condition, Treatment('Salmonella'))                                  │
└────────────────────────────────────────┴────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Intercepts                                                                                                      │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                        Final Parameter  Expected Sample                                                         │
│ Cell Type                                                                                                       │
│ Endocrine                  1.222            28.620                                                              │
│ Enterocyte                 3.650           324.440                                                              │
│ Enterocyte.Progenitor      2.523           105.120                                                              │
│ Goblet                     1.729            47.518                                                              │
│ Stem                       2.578           111.064                                                              │
│ TA                         1.995            61.998                                                              │
│ TA.Early                   2.835           143.610                                                              │
│ Tuft                       0.404            12.630                                                              │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Effects                                                                                                         │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                                                                       Final Parameter  Expected Sample          │
│ log2-fold change                                                                                                │
│ Covariate                                      Cell Type                                                        │
│ C(condition, Treatment('Salmonella'))T.Control Endocrine                   0.000           40.092               │
│ 0.486                                                                                                           │
│                                                Enterocyte                 -1.333          119.788               │
│ -1.437                                                                                                          │
│                                                Enterocyte.Progenitor       0.000          147.256               │
│ 0.486                                                                                                           │
│                                                Goblet                      0.000           66.565               │
│ 0.486                                                                                                           │
│                                                Stem                        0.000          155.582               │
│ 0.486                                                                                                           │
│                                                TA                          0.000           86.849               │
│ 0.486                                                                                                           │
│                                                TA.Early                    0.000          201.175               │
│ 0.486                                                                                                           │
│                                                Tuft                        0.000           17.693               │
│ 0.486                                                                                                           │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

### Compare

In [29]:
result_df_old = switch_results.effect_df
result_df_new = sccoda_model.get_effect_df(sccoda_data["coda_salm"])

result_df_combined = pd.DataFrame({
    "Final Parameter (old)": result_df_old["Final Parameter"].values,
    "Final Parameter (new)": result_df_new["Final Parameter"],
    "log2-fold change (old)": result_df_old["log2-fold change"].values,
    "log2-fold change (new)": result_df_new["log2-fold change"],
},index=result_df_new.index)
result_df_combined

Final Parameter (old)  \
Covariate                                      Cell Type                                      
C(condition, Treatment('Salmonella'))T.Control Endocrine                           0.000000   
                                               Enterocyte                         -1.340024   
                                               Enterocyte.Progenitor               0.000000   
                                               Goblet                              0.000000   
                                               Stem                                0.000000   
                                               TA                                  0.000000   
                                               TA.Early                            0.000000   
                                               Tuft                                0.000000   

                                                                      Final Parameter (new)  \
Covariate                                      Cell Type                                      
C(condition, Treatment('Salmonella'))T.Control Endocrine                           0.000000   
                                               Enterocyte                         -1.333453   
                                               Enterocyte.Progenitor               0.000000   
                                               Goblet                              0.000000   
                                               Stem                                0.000000   
                                               TA                                  0.000000   
                                               TA.Early                            0.000000   
                                               Tuft                                0.000000   

                                                                      log2-fold change (old)  \
Covariate                                      Cell Type                                       
C(condition, Treatment('Salmonella'))T.Control Endocrine                            0.489783   
                                               Enterocyte                          -1.443464   
                                               Enterocyte.Progenitor                0.489783   
                                               Goblet                               0.489783   
                                               Stem                                 0.489783   
                                               TA                                   0.489783   
                                               TA.Early                             0.489783   
                                               Tuft                                 0.489783   

                                                                      log2-fold change (new)  
Covariate                                      Cell Type                                      
C(condition, Treatment('Salmonella'))T.Control Endocrine                            0.486292  
                                               Enterocyte                          -1.437474  
                                               Enterocyte.Progenitor                0.486292  
                                               Goblet                               0.486292  
                                               Stem                                 0.486292  
                                               TA                                   0.486292  
                                               TA.Early                             0.486292  
                                               Tuft                                 0.486292

## Different reference cell type

### Old implementation

In [43]:
# switching reference cell type
model_salm_ref = mod.CompositionalAnalysis(
    data_salm, formula="condition", reference_cell_type="Enterocyte"
)
reference_results = model_salm_ref.sample_hmc()
reference_results.set_fdr(est_fdr=0.4)
reference_results.summary()

100%|██████████| 20000/20000 [00:26<00:00, 764.21it/s]


MCMC sampling finished. (33.535 sec)
Acceptance rate: 50.5%
Compositional Analysis summary:

Data: 6 samples, 8 cell types
Reference index: 1
Formula: condition

Intercepts:
                       Final Parameter  Expected Sample
Cell Type                                              
Endocrine                        0.573        34.851550
Enterocyte                       2.096       159.827901
Enterocyte.Progenitor            1.878       128.521682
Goblet                           1.105        59.328964
Stem                             2.109       161.919228
TA                               1.490        87.190695
TA.Early                         2.238       184.213907
Tuft                            -0.026        19.146073


Effects:
                                               Final Parameter  \
Covariate               Cell Type                                
condition[T.Salmonella] Endocrine                     0.000000   
                        Enterocyte                    0.0

### New implementation

In [44]:
# switching reference cell type
sccoda_data = sccoda_model.prepare(
    sccoda_data,
    modality_key="coda_salm",
    formula="condition",
    reference_cell_type="Enterocyte",
)
sccoda_model.run_nuts(sccoda_data, modality_key="coda_salm")
sccoda_model.set_fdr(sccoda_data, modality_key="coda_salm", est_fdr=0.4)
sccoda_model.summary(sccoda_data, modality_key="coda_salm")

sample: 100%|██████████| 11000/11000 [00:08<00:00, 1264.33it/s, 63 steps of size 6.56e-02. acc. prob=0.74]


                                          Compositional Analysis summary                                           
┌──────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────┐
│ Name                                         │ Value                                                            │
├──────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────┤
│ Data                                         │ Data: 6 samples, 8 cell types                                    │
│ Reference cell type                          │ Enterocyte                                                       │
│ Formula                                      │ condition                                                        │
└──────────────────────────────────────────────┴──────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Intercepts                                                                                                      │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                        Final Parameter  Expected Sample                                                         │
│ Cell Type                                                                                                       │
│ Endocrine                   0.552           35.042                                                              │
│ Enterocyte                  2.073          160.382                                                              │
│ Enterocyte.Progenitor       1.859          129.484                                                              │
│ Goblet                      1.085           59.714                                                              │
│ Stem                        2.077          161.025                                                              │
│ TA                          1.457           86.622                                                              │
│ TA.Early                    2.209          183.747                                                              │
│ Tuft                       -0.061           18.983                                                              │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Effects                                                                                                         │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                                              Final Parameter  Expected Sample  log2-fold change                 │
│ Covariate             Cell Type                                                                                 │
│ conditionT.Salmonella Endocrine                   0.000           41.472             0.243                      │
│                       Enterocyte                  0.000          189.809             0.243                      │
│                       Enterocyte.Progenitor       0.000          153.242             0.243                      │
│                       Goblet                      0.000           70.670             0.243                      │
│                       Stem                       -0.441          122.582            -0.394                      │
│                       TA                         -0.344           72.672            -0.253                      │
│                       TA.Early                   -0.294          162.086            -0.181                      │
│                       Tuft                        0.000           22.466             0.243                      │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

### Compare

In [45]:
result_df_old = reference_results.effect_df
result_df_new = sccoda_model.get_effect_df(sccoda_data["coda_salm"])

result_df_combined = pd.DataFrame({
    "Final Parameter (old)": result_df_old["Final Parameter"].values,
    "Final Parameter (new)": result_df_new["Final Parameter"],
    "log2-fold change (old)": result_df_old["log2-fold change"].values,
    "log2-fold change (new)": result_df_new["log2-fold change"],
},index=result_df_new.index)
result_df_combined

Final Parameter (old)  \
Covariate             Cell Type                                      
conditionT.Salmonella Endocrine                           0.000000   
                      Enterocyte                          0.000000   
                      Enterocyte.Progenitor               0.000000   
                      Goblet                              0.000000   
                      Stem                               -0.473469   
                      TA                                 -0.379547   
                      TA.Early                           -0.322684   
                      Tuft                                0.000000   

                                             Final Parameter (new)  \
Covariate             Cell Type                                      
conditionT.Salmonella Endocrine                           0.000000   
                      Enterocyte                          0.000000   
                      Enterocyte.Progenitor               0.000000   
                      Goblet                              0.000000   
                      Stem                               -0.441236   
                      TA                                 -0.344070   
                      TA.Early                           -0.293891   
                      Tuft                                0.000000   

                                             log2-fold change (old)  \
Covariate             Cell Type                                       
conditionT.Salmonella Endocrine                            0.263545   
                      Enterocyte                           0.263545   
                      Enterocyte.Progenitor                0.263545   
                      Goblet                               0.263545   
                      Stem                                -0.419527   
                      TA                                  -0.284025   
                      TA.Early                            -0.201989   
                      Tuft                                 0.263545   

                                             log2-fold change (new)  
Covariate             Cell Type                                      
conditionT.Salmonella Endocrine                            0.243037  
                      Enterocyte                           0.243037  
                      Enterocyte.Progenitor                0.243037  
                      Goblet                               0.243037  
                      Stem                                -0.393532  
                      TA                                  -0.253351  
                      TA.Early                            -0.180959  
                      Tuft                                 0.243037